In [1]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple
import dsds.metrics as me
import dsds.prescreen as ps
import dsds.sample as sa

In [ ]:
df = pl.DataFrame({
    "a": [1, np.nan, None],
    "b":[1,2,3]
})
df

In [2]:
import dsds.transform as t
df = pl.DataFrame({
    "a": [1, 1, 0],
    "b":[1, 0, 1],
    "c":[1, 1, 1]
})
print(t.combine_zero_ones(df, cols=["a", "b", "c"], new_name="abc", rule="same"))

.when([([([(0) + (col("a"))]) + (col("b"))]) + (col("c"))].is_in([Series])).then(1.strict_cast(UInt8)).otherwise(0.strict_cast(UInt8)).alias("abc")
shape: (3, 1)
┌─────┐
│ abc │
│ --- │
│ u8  │
╞═════╡
│ 1   │
│ 0   │
│ 0   │
└─────┘


In [ ]:
ps.invalid_inferral(df, threshold=0.5, include_null=True)

In [ ]:
pred = np.random.random(size=1_000_000) # .reshape(-1, 1) # Predictions
actual = np.round(np.random.random(size=1_000_000)).astype(np.int8)

In [ ]:
actual

In [ ]:
from sklearn.datasets import make_classification
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 50, n_informative = 20, n_redundant = 30)
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))

In [ ]:

me.logloss(actual, pred, check_binary=False)